In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [3]:
np.random.seed(2018)

trn = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/train_ver2.csv')
tst = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/test_ver2.csv')

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## 데이터 전처리 ##

# 제품 변수
prods = trn.columns[24:].tolist()

# 제품 변수 결측값 0으로 대체
trn[prods] = trn[prods].fillna(0).astype(np.int8)

In [5]:
#24개 제품 중 하나도 보유하지 않은 고객 제거
no_product = trn[prods].sum(axis=1) ==0
trn = trn[~no_product]

In [6]:
# 훈련 데이터 테스트 데이터 통합 테스트 데이터에 없는 제품 변수 0으로 채움
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis = 0)

In [7]:
# 학습 변수 list
features = []


In [8]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp','canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']

In [9]:
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel = -99)
features += categorical_cols

In [10]:
df['renta'][929602].values

array([137940.75, '         NA'], dtype=object)

In [11]:
df.renta.unique()

array([87218.1, 35548.74, 122179.11000000002, ..., '  139164.12',
       '  100647.45', '   72765.27'], dtype=object)

In [12]:
# 수치형 변수의 특이값, 결측값 -99로 대체하고 정수형 변환
df['age'].replace(' NA', -99, inplace = True)
df['age']  = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace = True)
df['antiguedad']  = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace = True)
df['renta'].fillna(-99, inplace=True)
df['renta']  = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace = True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes']  = df['indrel_1mes'].astype(float).astype(np.int8)

In [13]:
# 학습에 사용되는 수치형 변수 features에 추가
features += ['age' ,'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

## Feature Engineering

In [14]:
# 날짜 변수에서 연도와 월 정보 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [15]:
# 그외 변수의 결측값은 -99로 대체
df.fillna(-99, inplace=True)

In [16]:
# lag-1 데이터 생성

# 날짜를 숫자로 변환
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date


# 날짜를 숫자로 변환해 int_date에 저장
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

#데이터 카피하고 , int_date 날짜에 1을 더해 lag를 생성, 변수명에 _prev 추가
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

In [17]:
#원본과 lag데이터를 ncodper와 int_date기준으로 합침
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how = 'left')

In [18]:
del df, df_lag

In [19]:
# 전 달의 제품 정보가 존재 하지 않을시 0으로 대체
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace = True)

df_trn.fillna(-99, inplace=True)

In [20]:
#lag - 1 변수 추가
features +=  [feature + '_prev' for feature in features]
features +=  [prod + '_prev' for prod in prods]


###
### 추가 피쳐 엔지니어링
###

In [21]:
features

['ind_empleado',
 'pais_residencia',
 'sexo',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'nomprov',
 'segmento',
 'age',
 'antiguedad',
 'renta',
 'ind_nuevo',
 'indrel',
 'indrel_1mes',
 'ind_actividad_cliente',
 'fecha_alta_month',
 'fecha_alta_year',
 'ult_fec_cli_1t_month',
 'ult_fec_cli_1t_year',
 'ind_empleado_prev',
 'pais_residencia_prev',
 'sexo_prev',
 'tiprel_1mes_prev',
 'indresi_prev',
 'indext_prev',
 'conyuemp_prev',
 'canal_entrada_prev',
 'indfall_prev',
 'tipodom_prev',
 'nomprov_prev',
 'segmento_prev',
 'age_prev',
 'antiguedad_prev',
 'renta_prev',
 'ind_nuevo_prev',
 'indrel_prev',
 'indrel_1mes_prev',
 'ind_actividad_cliente_prev',
 'fecha_alta_month_prev',
 'fecha_alta_year_prev',
 'ult_fec_cli_1t_month_prev',
 'ult_fec_cli_1t_year_prev',
 'ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_

## 교차검증 데이터 분리

In [22]:
# 2016-01-28 ~ 2016-04-28데이터를 학습에 사용하고
# 검증에 2016-05-28 데이터를 사용한다.

use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn




In [23]:
# 훈련 데이터에서 신규 구매 건수만 추출
X = []
Y = []

for i,prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0 )] # 상품 구매자
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i #상품 숫자(LE)
    X.append(prX)
    Y.append(prY)

In [24]:
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [25]:
# 훈련 검증 데이터 분리
vld_date = '2016-05-28'
xy_trn = XY[XY['fecha_dato'] != vld_date]
xy_vld = XY[XY['fecha_dato'] == vld_date]

## XGBoost 학습

In [27]:
param = {
    'booster' : 'gbtree',
    'max_depth' : 8,
    'nthread' : -1,
    'num_class' : len(prods),
    'objective' : 'multi:softprob',
    'silent' : 1,
    'eval_metric' : 'mlogloss',
    'eta' : 0.1,
    'min_child_weight' : 10,
    'colsample_bytree' : 0.8,
    'colsample_bylevel' : 0.9,
    'seed' : 2018,
    'max_bin': 16, # GPU 
    'tree_method': 'gpu_hist', # GPU method (자세한 설명은 문서로!)
    'predictor':'gpu_predictor' # train뿐만아니라 predict할때도 gpu쓸건지 결정
    
}
# XGBoost 형태로 훈련, 검증 데이터 변환
X_trn = xy_trn.as_matrix(columns=features)
Y_trn = xy_trn.as_matrix(columns=['y'])
dtrn = xgb.(X_trn, label=Y_trn, feature_names=features)

X_vld = xy_vld.as_matrix(columns=features)
Y_vld = xy_vld.as_matrix(columns=['y'])
dvld = xgb.(X_vld, label=Y_vld, feature_names=features)




SyntaxError: invalid syntax (<ipython-input-27-dfadb907a9a2>, line 22)

In [78]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20, verbose_eval=1)

XGBoostError: b'[18:59:02] src/learner.cc:151: XGBoost version not compiled with GPU support.\n\nStack trace returned 10 entries:\n[bt] (0) /home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/xgboost/./lib/libxgboost.so(_ZN4dmlc15LogMessageFatalD1Ev+0x3c) [0x7fe3a68dc44c]\n[bt] (1) /home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/xgboost/./lib/libxgboost.so(_ZN7xgboost11LearnerImpl17ConfigureUpdatersEv+0x179) [0x7fe3a68e7379]\n[bt] (2) /home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/xgboost/./lib/libxgboost.so(_ZN7xgboost11LearnerImpl9ConfigureERKSt6vectorISt4pairINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES8_ESaIS9_EE+0x4c5) [0x7fe3a68e9d05]\n[bt] (3) /home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x4e) [0x7fe3a6a7a8de]\n[bt] (4) /home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/lib-dynload/..